<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Full_Test_Jacobians_Resnets_SVD_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notebook for generating Full Jacobian SVD over the test data for ResNet data

Can do the full calculation, and a batched (slow, but handles large memory cases)

Can handle: ResnetXXX with CIFAR10, CIFAR100, SVHN

i.e. CIFAR 10
```
 resnet20_cifar10
 resnet56_cifar10
 resnet110_cifar10
 resnet164bn_cifar10
 resnet272bn_cifar10

full_jacobian()


 resnet542bn_cifar10
 resnet1001_cifar10
 resnet1202_cifar10

full_batched_jacobian()

```






In [1]:
Jdir =  '/content/drive/My Drive/J_resnets_fullsvd'

thismodel = 'resnet1001_cifar100'
num_classes= 100
num_data = 10000

batched = True
start_batch = 0
batch_size = 50

#device = 'cuda:0'
#dev = xm.xla_device()



In [2]:
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4264  100  4264    0     0  78962      0 --:--:-- --:--:-- --:--:-- 78962
Updating TPU and VM. This may take around 2 minutes.
Updating TPU runtime to pytorch-dev20200515 ...
Uninstalling torch-1.6.0a0+bf2bbd9:
  Successfully uninstalled torch-1.6.0a0+bf2bbd9
Uninstalling torchvision-0.7.0a0+a6073f0:
  Successfully uninstalled torchvision-0.7.0a0+a6073f0
Copying gs://tpu-pytorch/wheels/torch-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
\ [1 files][ 91.0 MiB/ 91.0 MiB]                                                
Operation completed over 1 objects/91.0 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-nightly+20200515-cp36-cp36m-linux_x86_64.whl...
\ [1 files][119.5 MiB/119.5 MiB]                                                
Operation completed over 1 objects/119.5 MiB.               

In [3]:
# imports pytorch
import torch
# imports the torch_xla package
import torch_xla
import torch_xla.core.xla_model as xm
device = xm.xla_device()
print(device)

xla:1


In [4]:
!pip install ipython-autotime
%load_ext autotime

In [5]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
time: 1.25 ms


In [6]:
!pip install pytorchcv
!pip install powerlaw


time: 5 s


In [7]:
import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy

import gc


[('__call__', <function LevelMapper.__call__ at 0x7fcf77f8eb70>), ('__init__', <function LevelMapper.__init__ at 0x7fcf77f8eae8>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x7fcf75a4b730>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x7fcf75a4b6a8>)]
[('__init__', <function BoxCoder.__init__ at 0x7fcf75a63c80>), ('decode', <function BoxCoder.decode at 0x7fcf75a63e18>), ('decode_single', <function BoxCoder.decode_single at 0x7fcf75a63ea0>), ('encode', <function BoxCoder.encode at 0x7fcf75a63d08>), ('encode_single', <function BoxCoder.encode_single at 0x7fcf75a63d90>)]
[('__call__', <function Matcher.__call__ at 0x7fcf75a4b9d8>), ('__init__', <function Matcher.__init__ at 0x7fcf75a4b8c8>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x7fcf75a63b70>)]
[('__init__', <function ImageList.__init__ at 0x7fcf75a7a048>), ('to', <function ImageList.to at 0x7fcf75a7a0d0>)]
[('__init__', <function Timebase.__init__ at

### Run InstallPhysTheory notebook 

In [8]:
import sys
sys.path.append("/content/drive/My Drive/Jacobian")
import jacobian as jac
import utils

time: 7.87 ms


In [9]:
import torch
from torch.autograd.gradcheck import zero_gradients
import numpy as np
import math

def batch_diagJ(inputs, output):
  """Computes the diagonal Jacobian by input batches.
  Input: Input for the function for which the Jacobian will
  computed. It will be batch_size*data_dim. Make sure that the
  input is flagged as requires_grad=True with the torch.autograd.Variable
  wrapper. 
  Output: Output of the function for which the Jacobian will
  be computed. It will be batch_size*classes
  Return: Jacobian of dimension batch_size*classes*data_dim
  """
  assert inputs.requires_grad
  print("inside batch diagJ")

  num_classes = output.size()[1] #0 index is batch
  print("num classes", num_classes)

  jacobian = torch.zeros(num_classes, *inputs.size())
  grad_output = torch.zeros(*output.size())
  print(type(jacobian))
  print(device)
  grad_output = grad_output.to(device)
  jacobian = jacobian.to(device)
  print(type(jacobian))

  #zero out gradients
  #compute gradient for one of the classes
  for i in range(num_classes):
    print("i", i)

    zero_gradients(inputs)
    grad_output.zero_()
    grad_output[:,i] = 1
    output.backward(grad_output, retain_graph=True)
    jacobian[i] = inputs.grad.data

  return torch.transpose(jacobian, dim0=0, dim1=1)

time: 12.4 ms


In [10]:
def jacobian_batched_full(modelname, model, data_loader, batch_size, start_batch = 0, num_classes=10, device=device, data_dim=3*32*32, formJJ=False, save=True): 
  '''Compute a full of J^{T}J Jacobian in batch mode.  '''

  model.eval() 
  model = model.to(device)

  left_data_loader = data_loader 
  right_data_loader = copy.deepcopy(data_loader)

  JJMat = None 
  for left_batch, data in enumerate(left_data_loader): 
    if left_batch < start_batch:
      continue
    print(left_batch) 

    inputs, _ = data 
    inputs = inputs.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)
    J = batch_diagJ(inputs, outputs)
    J = J.reshape(batch_size,num_classes*data_dim)

    X = J.detach().cpu().numpy()

    del data, inputs, outputs, J
    gc.collect()
    #torch.cuda.empty_cache()
    #GPUtil.showUtilization()

    JJBlock = None
    for right_batch, data in enumerate(right_data_loader):
      print(left_batch, right_batch)
      inputs, _ = data 
      inputs = inputs.to(device)
      inputs.requires_grad=True
      outputs = model(inputs)
    
      Jt = batch_diagJ(inputs, outputs)
      Jt = Jt.reshape(batch_size,num_classes*data_dim)

      Xt = Jt.detach().cpu().numpy().transpose()

      del data, inputs, outputs, Jt
      gc.collect()
      #torch.cuda.empty_cache()

      block = np.dot(X,Xt)
      JJBlock = np.hstack([JJBlock, block]) if JJBlock is not None else block

    # end of right_batch loop
    
    if save:
      filename = "{}/Jfull_{}__{}.csv".format(Jdir, modelname, left_batch)
      np.savetxt(filename, JJBlock, delimiter='\t')
      print("Saved file ",filename)

    if formJJ:
      JJMat = np.vstack([JJMat, JJBlock]) if JJMat is not None else JJBlock

      print("JJMat shape ", left_batch, JJMat.shape, filename) 

  # end of left_batch loop

  return JJMat


time: 40.4 ms


### ResNet 

In [11]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

time: 109 ms


### Compute JJ, in batches or all at once


In [12]:
def get_datasets(thismodel, batch_size):
  print("getting dataset for ", thismodel)
  if thismodel.endswith('cifar100'):
    print("loading cifar100")
    train_dataset = datasets.CIFAR100(
      root='data', 
      train=True, 
      transform=transforms.ToTensor(),
      download=True)
    test_dataset = datasets.CIFAR100(
      root='data', 
      train=False, 
      transform=transforms.ToTensor(),
      download=True) 
 
    train_loader = DataLoader(
      dataset=train_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)
    test_loader = DataLoader(
      dataset=test_dataset, 
      batch_size=batch_size,
      num_workers=4,
      shuffle=False)

  else:
    print("loading cifar10")
    train_dataset, test_dataset, train_loader, test_loader = utils.get_data(batch_size=batch_size)

      
  return train_dataset, test_dataset, train_loader, test_loader

time: 7.51 ms


In [13]:
print(thismodel)
train_dataset, test_dataset, train_loader, test_loader = get_datasets(thismodel, batch_size)


resnet1001_cifar100
getting dataset for  resnet1001_cifar100
loading cifar100
Files already downloaded and verified
Files already downloaded and verified
time: 2.04 s


In [ ]:
model = ptcv_get_model(thismodel, pretrained=True)  
if batched: 
  jacobian_batched_full(thismodel, model, test_loader, batch_size=batch_size, num_classes=num_classes, start_batch=start_batch)
else:
  JJMat = jacobian_full(model, test_loader, batch_size=batch_size,  num_classes=num_classes,)

0
inside batch diagJ
num classes 100
<class 'torch.Tensor'>
xla:1


In [ ]:
if batched:
  print("Reading batched JJBlocks for ", thismodel)
  JJMat = None
  import os, glob
  os.chdir(Jdir)
  for filename in glob.glob("Jfull_{}*".format(thismodel)):
    print(filename)
    JJBlock = np.loadtxt(filename)
    JJMat = np.vstack([JJMat, JJBlock]) if JJMat is not None else JJBlock


In [4]:
import torch 

jacobian = torch.zeros(num_classes, 1000)
grad_output = torch.zeros(100)
print(type(jacobian))
print(device)
grad_output = grad_output.to(device)
jacobian = jacobian.to(device)
print(type(jacobian))


NameError: ignored

In [ ]:
import powerlaw
norm = num_classes*3*32*32
evals = np.linalg.eigh(JJMat)
evals  = (1.0/ norm)*evals

fit = powerlaw.Fit(evals, xmin=0.01)
alpha = fit.alpha
D = fit.D
max_eval = np.max(evals)

plt.hist(evals, bins=100)
plt.title("{}  alpha = {:0.2f} max  = {:0.2f} ".format(thismodel, alpha, max_eval))
plt.show()

print(alpha, D, max_eval)
